_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [ ]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [ ]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

In [ ]:
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



In [ ]:
df.sample(10)

In [ ]:
# Check Pandas Profiling version
import pandas_profiling
pandas_profiling.__version__

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, minimal=True)

profile.to_notebook_iframe()

## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [ ]:
y_val = df['shot_made_flag']
y_val.value_counts(normalize=True)

In [ ]:
majority = y_val.mode()[0]
y_pred = [majority]*len(y_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [ ]:
df['game_date'] = pd.to_datetime(df['game_date'], infer_datetime_format=True)

In [ ]:
#train, val, test
cutoff = pd.to_datetime('2018-07-01')
train = df[df['game_date'] < cutoff]
test  = df[df['game_date'] >= cutoff]

train.shape, test.shape

## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [ ]:
df.head(2)

In [ ]:
def engineer_features(df):
    
    df = df.copy()
    
    # homecourt advantage
    df['home_advantage'] = (df['htm'] == 'GSW')
    
    # Opponent
    # df['opponent'] = (df['htm'] != 'GSW') | (df['vtm'] != 'GSW')
    # this didn't extract the value.
    
    # Seconds remaining in period 
    df['seconds_remaining_in_period'] = (df['minutes_remaining']*60) +(df['seconds_remaining'])
    
    # Drop columns that were used for engineering
    # df = df.drop(columns='htm')
    # df = df.drop(columns='vtm')
    df = df.drop(columns='minutes_remaining')
    df = df.drop(columns='seconds_remaining')
    df = df.drop(columns='game_date')
    df = df.drop(columns='game_id')
    df = df.drop(columns='game_event_id')
    df = df.drop(columns='player_name')
    
    return df

train = engineer_features(train)
test = engineer_features(test)

In [ ]:
train.sample(10)

In [ ]:
train.dtypes

## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [ ]:
target = 'shot_made_flag'
features = train.columns.drop([target])
X_train = train[features]
y_train = train[target]

## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [ ]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_jobs=-1, random_state=7, max_depth=20)
)

In [ ]:
from scipy.stats import randint, uniform

param_distributions = {
    'simpleimputer__strategy':['mean'],
    'randomforestclassifier__max_depth':[5, 10, 15, 20, None],
    'randomforestclassifier__n_estimators': randint(50,500)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=15,
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

In [ ]:
print('Best hyperparameters', search.best_params_)

In [ ]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [ ]:
scores = cross_val_score(X_train, y_train, cv=5)

## 7. Get your model's test accuracy

> (One time, at the end.)

In [ ]:
print('Test accuracy:', -search.best_score_)

In [ ]:
pipeline = search.best_estimator_

In [ ]:
from sklearn.metrics import mean_absolute_error

X_test = test[features]
y_test = test[target]

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE: {mae}')

## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [ ]:
# !pip install scikit-plot

In [ ]:
#from scikitplot.metrics import plot_confusion_matrix
#plot_confusion_matrix(y_test, y_pred, figsize=(20,10),title=f"Confusion Matric: N={len(y_test)}",normalize=False);
# I was using the wrong data for this

In [ ]:
accuracy = (85+36)/(85+58+8+36)
print(accuracy)

### Calculate precision

In [ ]:
neg_predict = 85+8
pos_predict = 58+36
correct_pos = 36
correct_neg = 85

neg_precision = correct_neg / neg_predict
pos_precision = correct_pos / pos_predict

print('Precision for negative predictions:', neg_precision)
print('Precision for positive predictions:', pos_precision)

### Calculate recall

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))
# I was using the wrong data for this

In [ ]:
neg_actual = 85+58
pos_actual = 8+36

neg_recall = correct_neg / neg_actual
pos_recall = correct_pos / pos_actual

print('Recall for negative predictions:', neg_recall)
print('Recall for positive predictions:', pos_recall)